In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Correction factor fIMF for the sampling from a Kroupa IMF

## Functions from script of Giuliano to calculate IMF correction factor

In [2]:
def Q(qmin,qmax):
    
    A=(qmax**0.9-qmin**0.9)/0.9
    B=(qmax**1.9-qmin**1.9)/1.9
    
    return A+B

def Mintegral(Mmin,Mmax):
    
    Mmin=np.atleast_1d(Mmin)
    Mmax=np.atleast_1d(Mmax)
    
    if (len(Mmin)==1 and len(Mmax)>1): Mmin=np.array([Mmin[0],]*len(Mmax))
    elif (len(Mmax)==1 and len(Mmin)>1): Mmax=np.array([Mmax[0],]*len(Mmin))
    
    Mblow=0.08
    Mbint=0.5
    
    fMlow           = lambda m1,m2: (25/1.7)*(m2**1.7 - m1**1.7)
    fMintermediate  = lambda m1,m2: (2/0.7)*(m2**0.7 - m1**0.7)
    fMhigh          = lambda m1,m2: (-1/0.3)*(m2**(-0.3) - m1**(-0.3))
    
    ret=np.zeros_like(Mmin)
    ret=np.where(Mmax>=Mmin,0,np.nan)
    
    #Case-1, Mmin<Mmax<Mblow
    idx=(Mmin<Mblow) & (Mmax<=Mblow)
    ret[idx]=fMlow(Mmin[idx],Mmax[idx])
    
    #Case-2, Mmin<Mblow<Mmax<Mbint
    idx=(Mmin<Mblow) & (Mmax<=Mbint)
    ret[idx]= fMlow(Mmin[idx],Mblow)  + fMintermediate(Mblow,Mmax[idx])
    
    #Case-3, Mmin<Mblow<Mbint<Mmax
    idx=(Mmin<Mblow) & (Mmax>Mbint)
    ret[idx]= fMlow(Mmin[idx],Mblow) +  fMintermediate(Mblow,Mbint)  + fMhigh(Mbint,Mmax[idx])
    
    #Case-4, Mblow<Mmin<Mmax<Mbint
    idx=(Mmin>=Mblow) & (Mmin<Mbint) & (Mmax<=Mbint)
    ret[idx]=  fMintermediate(Mmin[idx],Mmax[idx]) 
    
    #Case-5, Mblow<Mmin<Mbint<Mmax
    idx=(Mmin>=Mblow) & (Mmin<Mbint) & (Mmax>Mbint)
    ret[idx]=  fMintermediate(Mmin[idx],Mbint)  + fMhigh(Mbint,Mmax[idx])
    
    #Case-5, Mbint<Mmin<Mmax
    idx=(Mmin>=Mbint)  & (Mmax>Mbint)
    ret[idx]= fMhigh(Mmin[idx],Mmax[idx])

    
    return ret

def Mpop_independent(M1min,M1max,qmin,qmax):
    
    Mcal = Mintegral(M1min,M1max)
    Qcal = Q(qmin,qmax)
    
    return Mcal*Qcal

def fIMF_independent(M1min,M1max,qmin,qmax,M1ming=0.1,M1maxg=150,qming=0.1,qmaxg=1):
    
    #Check inputs
    if (np.any(qmin<qming)): raise ValueError(f"qmin  cannot be smaller than the qmin of the parent population ")
    elif (np.any(qmax>qmaxg)): raise ValueError(f"qmax  cannot be larger than the qmax of the parent population")
    elif (np.any(M1min<M1ming)): raise ValueError(f"M1min  cannot be smaller than the M1min of the parent population ")
    elif (np.any(M1max>M1maxg)):  raise ValueError(f"M1max  cannot be larger than the M1max of the parent population")
    
    Msubpop = Mpop_independent(M1min,M1max,qmin,qmax)
    Mpop    = Mpop_independent(M1ming,M1maxg,qming,qmaxg)
    
    return Msubpop/Mpop

We sampled $5 \times 10^6$ binaries from a Kroupa IMF with 

$M_1 \in [5,150] M_\odot$
$q \in [0.1,1]$

Is a CASE-A type

In [3]:
M1min = 5
fIMF = fIMF_independent(M1min,150,0.1,1)[0]
print(f'The correction factor for Kroupa IMF sampled with a minimum mass {M1min} cut is fIMF={fIMF}')

The correction factor for Kroupa IMF sampled with a minimum mass 5 cut is fIMF=0.2890237434629212


# Total mass simulated
We consider the total mass of the $5 \times 10^6$ sample. Even if the parsec tracks limit the real evolution to only $\sim 4 180 000$ systems (the ones in listBin5e6seed.dat), those $M_{ZAMS} \leq 2.2 M_\odot$ would not have produced a BH, so won't enter in the formation efficiency of BBH or MS-BH systems (in other words, their mass budget is not a viable progenitor budget to produce the objects we are interested in).

Also, note that in each set some systems ($\sim 100-1000$) fail because they merge after that their semimajor axis reduces below the helium core size. In the 2.15 version of SEVN these systems do not fail anymore, but they still do in the version adopted in this work (after the commits of 25 August). These systems will not form a BBH anyway, since they merge, so also their mass budget will be ignored for BBH formation. We also manually checked that these systems would form MS-BH, if any, at low periods, below the region allowed for BBH formation; where premature mergers occur (indeed). So if we were to consider also these systems, we expect a slight increase in the formation efficiency of MS-BH that coalesce, although the increase would be neglibigle (<0.1-1 \%).

To correct the simulated mass, first we calculate the mass generated from the Kroupa IMF ($M_{IC}$), then we correct it for the $M_{1, min}=5 M_\odot$ cut in the IMF. Doing so, we obtain the total mass $M_{tot}$ of the underlying population, without biases introduced by the IMF cuts

$M_{tot} = \frac{M_{IC}}{f_{IMF}}$

In [17]:
# read file with simulated mass extracted from evolved files
# with the mass_simulated.py script

sevn_version = 'sevn_tides_fix6Agodadt'
sims = ['IsoT','JeansT','LminT','LmaxT',
        'IsoNT','JeansNT','LminNT','LmaxNT']
path_to_df = f'./v_{sevn_version}/mass_simulated'
mass_simulated = pd.read_csv(f'{path_to_df}/mass_simulated.csv', sep=',').drop("Unnamed: 0", axis=1).set_index(pd.Index(sims))
print(mass_simulated)

              0.00142      0.000142
IsoT     1.008797e+08  1.008759e+08
JeansT   1.008793e+08  1.008761e+08
LminT    1.008794e+08  1.008758e+08
LmaxT    1.008793e+08  1.008757e+08
IsoNT    1.008772e+08  1.008692e+08
JeansNT  1.008772e+08  1.008692e+08
LminNT   1.008772e+08  1.008696e+08
LmaxNT   1.008764e+08  1.008696e+08


In [18]:
M_tot = mass_simulated / fIMF
print(M_tot)

              0.00142      0.000142
IsoT     3.490359e+08  3.490229e+08
JeansT   3.490348e+08  3.490235e+08
LminT    3.490349e+08  3.490225e+08
LmaxT    3.490347e+08  3.490223e+08
IsoNT    3.490274e+08  3.489998e+08
JeansNT  3.490274e+08  3.489996e+08
LminNT   3.490274e+08  3.490009e+08
LmaxNT   3.490245e+08  3.490012e+08


In [19]:
M_tot.to_csv(f'{path_to_df}/mass_corrected.csv')

In [4]:
# read file with simulated mass extracted from evolved files
# with the mass_simulated.py script

sevn_version = 'sevn_25ago'
sims = ['LmaxT','LmaxNT']
path_to_df = f'./v_{sevn_version}/mass_simulated'
mass_simulated = pd.read_csv(f'{path_to_df}/mass_simulated.csv', sep=',').drop("Unnamed: 0", axis=1).set_index(pd.Index(sims))
print(mass_simulated)

             0.00142      0.000142
LmaxT   1.006146e+08  1.007980e+08
LmaxNT  1.007459e+08  1.008585e+08


In [5]:
M_tot = mass_simulated / fIMF
print(M_tot)

             0.00142      0.000142
LmaxT   3.481189e+08  3.487535e+08
LmaxNT  3.485729e+08  3.489626e+08


In [6]:
M_tot.to_csv(f'{path_to_df}/mass_corrected.csv')

Note that original listBin5e6seed.dat file with $\sim$ 4 180 000 binaries had a mass of $M_{IC} \approx 1.07^8 M_\odot$, 

that would have corresponded to a corrected underlying mass of $M_{tot} \sim 3.72^8 M_\odot$